# DS-610 Week 3 Homework: Hive on Databricks
This week we will look at data can be manipulated on Apache Hive. As an example, we will look at a part of Walmart's daily stock prices.

For this assignment, it is highly recommended that you try it on Saint Peters' Databricks system to get a feel of how you can retrieve data from Hive. We will perform aggregations using the following three methods:
1. `groupBy` then `agg` syntax on the loaded DataFrame.
2. Creating a table view via `createOrReplaceTempView` then running SQL query on the table view.
3. `groupBy` then `agg` with a slightly complicated aggregation logic using a UDF.

First we start by loading the necessary libraries.

In [0]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.dataframe import DataFrame
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Data Source
By default the assignment is designed to run on Saint Peters' Databricks cluster. If you are running on your laptop, uncomment the line below. You also need to download the data file on the Blackboard website into the folder containing this Jupyter notebook.

In [0]:
# If you are running on Saint Peters' Databricks, then use the following line. If not, you need to adjust the file path to a local one.
from pyspark.sql.types import StringType, StructType, DateType, FloatType, StructField
fields = [ StructField('Date', DateType(), True),
    StructField('Open', FloatType(), True),
    StructField('High', FloatType(), True),
    StructField('Low', FloatType(), True),
    StructField('Close', FloatType(), True),
    StructField('Volume', FloatType(), True),
    StructField('Adj Close', FloatType(), True) ]
schema = StructType(fields)
df = spark.read.csv('dbfs:/FileStore/shared_uploads/dlee5@saintpeters.edu/ds610/walmart_stock.csv', schema=schema, header=True)

## Part 1: Warmup
Display the schema of the loaded DataFrame `df` via `printSchema` method.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.printSchema.html

In [0]:
# Your code for Part 1 goes here.
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- Adj Close: float (nullable = true)



## Part 2: Warmup
Display the first 15 rows of the DataFrame `df` via `show` method.

https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.DataFrame.show.html

As you will see, this is a time-stamped data with each data containing the opening price, the high price, the low price, the closing price with the trading volume. For our assignment `Adj Close` (adjusted closing price) is not considered. For more information:
https://www.investopedia.com/terms/a/adjusted_closing_price.asp

In [0]:
# Your code for Part 2 goes here.
df.show(15)

+----------+-----+-----+-----+-----+---------+---------+
|      Date| Open| High|  Low|Close|   Volume|Adj Close|
+----------+-----+-----+-----+-----+---------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|1.26688E7|52.619236|
|2012-01-04|60.21|60.35|59.47|59.71|9593300.0|52.078476|
|2012-01-05|59.35|59.62|58.37|59.42|1.27682E7| 51.82554|
|2012-01-06|59.42|59.45|58.87| 59.0|8069400.0| 51.45922|
|2012-01-09|59.03|59.55|58.92|59.18|6679300.0|51.616215|
|2012-01-10|59.43|59.71|58.98|59.04|6907300.0| 51.49411|
|2012-01-11|59.06|59.53|59.04| 59.4|6365600.0|51.808098|
|2012-01-12|59.79| 60.0| 59.4| 59.5|7236400.0|51.895317|
|2012-01-13|59.18|59.61|59.01|59.54|7729300.0|51.930202|
|2012-01-17|59.87|60.11|59.52|59.85|8500000.0| 52.20058|
|2012-01-18|59.79|60.03|59.65|60.01|5911400.0| 52.34013|
|2012-01-19|59.93|60.73|59.75|60.61|9234600.0|52.863445|
|2012-01-20|60.75|61.25|60.67|61.01|1.03788E7|53.212322|
|2012-01-23|60.81|60.98|60.51|60.91|7134100.0|53.125103|
|2012-01-24|60.75| 62.0|60.75|6

## Part 3: Apply Manipulation on DataFrame
Next, we decide that we would like to analyze data on the per-month basis and it would be helpful to have a column named `Month` in the DataFrame.

Write a code that creates another DataFrame `df2` with the same schema as `df` but an additional column called `Month`. For example, for a row in `df` with `Date=2012-01-03`, `Month` should be 1.

Although you can do this in many ways, it is recommended to use the `select` command:
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html

After you create `df2`, run `show` on it to quickly check whether it is working.

In [0]:
# Your code for Part 3 goes here.
from pyspark.sql.functions import year, month, dayofmonth
df2 = df.select('*', month(df.Date).alias('Month'))
df2.show()

+----------+-----+-----+-----+-----+---------+---------+-----+
|      Date| Open| High|  Low|Close|   Volume|Adj Close|Month|
+----------+-----+-----+-----+-----+---------+---------+-----+
|2012-01-03|59.97|61.06|59.87|60.33|1.26688E7|52.619236|    1|
|2012-01-04|60.21|60.35|59.47|59.71|9593300.0|52.078476|    1|
|2012-01-05|59.35|59.62|58.37|59.42|1.27682E7| 51.82554|    1|
|2012-01-06|59.42|59.45|58.87| 59.0|8069400.0| 51.45922|    1|
|2012-01-09|59.03|59.55|58.92|59.18|6679300.0|51.616215|    1|
|2012-01-10|59.43|59.71|58.98|59.04|6907300.0| 51.49411|    1|
|2012-01-11|59.06|59.53|59.04| 59.4|6365600.0|51.808098|    1|
|2012-01-12|59.79| 60.0| 59.4| 59.5|7236400.0|51.895317|    1|
|2012-01-13|59.18|59.61|59.01|59.54|7729300.0|51.930202|    1|
|2012-01-17|59.87|60.11|59.52|59.85|8500000.0| 52.20058|    1|
|2012-01-18|59.79|60.03|59.65|60.01|5911400.0| 52.34013|    1|
|2012-01-19|59.93|60.73|59.75|60.61|9234600.0|52.863445|    1|
|2012-01-20|60.75|61.25|60.67|61.01|1.03788E7|53.212322

### Interlude
The following code block will stop you from proceeding to the rest of the parts until the parts above are finished.

In [0]:
if not(df2 and type(df2) is DataFrame and 'Month' in df2.columns):
    raise ValueError("You will need to finish Part 1 before moving onto Part 2.")

In [0]:
df2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Month: integer (nullable = true)



## Part 4
We would like to find out for each month, the minimum, the average, and maximum closing prices.

Your task is to write the code to find out these information. You may find `groupBy` and `agg` syntax useful:
https://spark.apache.org/docs/latest/api/python//reference/pyspark.pandas/api/pyspark.pandas.groupby.DataFrameGroupBy.agg.html#pyspark.pandas.groupby.DataFrameGroupBy.agg

In [0]:
# Your code for Part 4 goes here.
from pyspark.sql.functions import sum, avg, min, max

df2.groupBy("Month").agg(min("Close").alias('Min_Closing_Price'),avg("Close").alias('Avg_Closing_Price'),max("Close").alias('Max_Closing_Price')).orderBy("Month").show()

+-----+-----------------+------------------+-----------------+
|Month|Min_Closing_Price| Avg_Closing_Price|Max_Closing_Price|
+-----+-----------------+------------------+-----------------+
|    1|             59.0|60.235499954223634|            61.47|
|    2|            58.46|60.897999954223636|            62.48|
|    3|            58.82|60.433636752041906|            61.23|
|    4|            57.36| 60.14900016784668|            62.45|
|    5|             58.7| 60.54388851589627|            65.07|
+-----+-----------------+------------------+-----------------+



## Part 5
Now let us do Part 4 but with SQL. The first thing we will do is to create a temporary table view on `df2`. This is done below for you.

Your task is now to write a SQL `SELECT` statement on the table `walmart_stock_price_history` to get the same results as Part 4. For documentation, see:

https://spark.apache.org/docs/latest/sql-getting-started.html#running-sql-queries-programmatically

In [0]:
# Create Temporary table in PySpark. DO NOT MODIFY.
df2.createOrReplaceTempView("walmart_stock_price_history")

In [0]:
# Your code for Part 5 goes here.

spark.sql("select Month,min(Close) as Min_Closing_Price ,avg(Close) as Avg_Closing_Price,max(Close) as Max_Closing_Price from walmart_stock_price_history group by Month order by Month").show()

+-----+-----------------+------------------+-----------------+
|Month|Min_Closing_Price| Avg_Closing_Price|Max_Closing_Price|
+-----+-----------------+------------------+-----------------+
|    1|             59.0|60.235499954223634|            61.47|
|    2|            58.46|60.897999954223636|            62.48|
|    3|            58.82|60.433636752041906|            61.23|
|    4|            57.36| 60.14900016784668|            62.45|
|    5|             58.7| 60.54388851589627|            65.07|
+-----+-----------------+------------------+-----------------+



## Part 6 (Extra Credit)
Now let us practice UDFs (https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.udf.html).

UDF's (User Defined Functions) support re-using transformations as a function which can registered in Spark environment.

Your task is to compute for each month, the median trading volume when the closing price is higher than the opening price. Note that this part has a slight *twist* - if you end up spending a lot of time on this part, you may want to skip.

For helpful examples, you may also want to look at:
https://danvatterott.com/blog/2018/09/06/python-aggregate-udfs-in-pyspark/

In [0]:
# Fill out your code for Part 6 in this box.
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
import statistics

# Your code for median volume computation goes here. The input to this function 
# would be a list of trading volumes for each month when the closing price is 
# higher than the opening price.
def compute_median_volume(x):
    
    # IMPLEMENT ME FOR PART 6
    return float(x[int(len(x) / 2)])

# This code registers the function as a UDF.
compute_median_volume_udf = F.udf(compute_median_volume, FloatType())

# Write your code for performing the aggregation to display the median price for
# each month for days whose closing price > opening price.

df2.filter(df2.Close > df2.Open).groupBy('Month').agg(compute_median_volume_udf(F.expr("collect_list(Volume)")).alias("Median_Trading_Volume")).orderBy('Month').show()

+-----+---------------------+
|Month|Median_Trading_Volume|
+-----+---------------------+
|    1|            9234600.0|
|    2|            5173100.0|
|    3|            8251900.0|
|    4|            7957300.0|
|    5|            2.92924E7|
+-----+---------------------+



## Part 7
What is your favorite way of performing aggregations (`groupBy` syntax, `createOrReplaceTempView` then SQL, `groupby` aggregation with UDF)? Did you spend a lot of time on Part 6?

In [0]:
# Your response to Part 5 goes here.
print("Your response to Part 5: My favorite way of performing aggregations is by using createOrReplaceTempView, then SQL. In this way, I was able to achieve the aggregation a bit fast as it is easy to understand, and groupBy syntax is also an easy way to do the aggregation, but UDF took some time for me as I have good knowledge of SQL, but this UDF is a new concept where I have spent some time to understand the concept. After that, I have implemented my learning and achieved the result.")

Your response to Part 5: My favorite way of performing aggregations is by using createOrReplaceTempView, then SQL. In this way, I was able to achieve the aggregation a bit fast as it is easy to understand, and groupBy syntax is also an easy way to do the aggregation, but UDF took some time for me as I have good knowledge of SQL, but this UDF is a new concept where I have spent some time to understand the concept. After that, I have implemented my learning and achieved the result.
